In [3]:
# Extracting useful information from IMDb movie reviews dataset

# unable to conda install pyprind due to dependency issues, wait for upgrade
import pandas as pd
import os
labels = {'pos':1, 'neg':0} # integer class label for positive and negative
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path='./aclImdb/%s/%s' % (s, l) 
        for file in os.listdir(path): # list all file names
            with open(os.path.join(path, file), 'r') as infile:
                txt = infile.read()
                df = df.append([[txt, labels[l]]], ignore_index = True) # continuous index
df.columns = ['review', 'sentiment']

In [4]:
# Shuffle the dataframe
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('./movie_data.csv', index=False) # store in csv for convenience

In [5]:
df = pd.read_csv('./movie_data.csv')
df.head(3)

,review,sentiment
0,"Susie Q. is one of those rare, and sweet movie...",1
1,It starts out looking like it may be going som...,0
2,Two films are useful for scaring people to God...,0


In [ ]:
# Bag-of-words model
# 1. create vocabulary of unique tokens - e.g. words from the entire set of documents
# 2. construct feature vector from each document that contains the counts of how often each word occurs in
# that particular document